# Projektuppgift 1 - Kontinuerlig balk
*Tobias Ericsson, David Neinhardt*  
*2025-03-27*  
*MTM026*  

---

## Bibliotek Introducerade  
- **Pylab**: Innehåller bibliotek som numpy och plotlib som kan vara nödvändiga i uppgiften
- **Sympy**: Är biblioteket som kommer att användas för majoriteten av alla beräkningsmetoder.
- **MTM026**: Ett special tillämpat bibliotek med nödvändiga funktioner som underlättar beräkningarnas gång

In [3]:
import pylab as pl
import sympy as sp
from mtm026 import *

...

In [ ]:
EI, L1, L2, Q, P, t, a = sp.symbols('EI L1 L2 Q P t a')

a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12 = sp.symbols('a1:13')
r1,r2,r3,r4,r5,r6,r7,r8,r9,r10,r11,r12 = sp.symbols('r1:13')

K = pl.zeros(12,12)

...

In [ ]:
ke1 = Ke_balk(EI, L1)
ke2 = ke1
ke3 = 